In [2]:
from pprint import pprint

from main import SSS

In [3]:
sss = SSS()
sss.set_hyper_parameters(256)
sss.init_graph()
sss.init_sess()

In [4]:
for _ in range(5):
    sss.run_sess()

100.00% | acc-l 0.4726562 | acc-v 0.5000000 | acc-l-s 0.5273438 | acc-v-s 0.0000000 | loss-gen 3.6382053


In [ ]:
result = sss.debug(777)
for mode in ['wake', 'sleep']:
    print(result[mode]['sentence'])
    for i in range(3):
        print(result[mode]['label'][i])
    for i in range(3):
        print(result[mode]['valid'][i])

- 포스태깅 백터 추가해서 돌리기
- 몇퍼센트 손상시킬지 정해주고 로스펑션에 반영하는 것이 당연함

In [ ]:
import pandas as pd
import time

from gensim.models import Word2Vec
import tensorflow as tf


def get_sess():
    tf.reset_default_graph()
    graph = tf.get_default_graph()
    lt = tf.constant(
        pd.read_pickle('./data/lookup_table'),
        dtype=tf.float32
    )
    sen = tf.truncated_normal([4, 64, 512])
    batch = tf.map_fn(lambda word: tf.map_fn(
        lambda x: tf.cast(tf.argmin(
            tf.norm((lt-x), axis=1)
        ), dtype=tf.float32), word
    ), sen)
    sess = tf.Session()
    return batch, sess, graph


def batch2sens(batch):
    
    i2w = Word2Vec.load('./data/w2v_model').wv.index2word
    
    def get_sen(l):
        ws = ["<BLANK>" if i == len(i2w) else i2w[i] for i in l]
        sen = ' '.join(w.split('-')[0] for w in ws)
        return sen

    batch = batch.astype(int)
    sens = [get_sen(l) for l in batch]
    for sen in sens:
        print(sen)


batch, sess, graph = get_sess()
batch_ = sess.run(batch)
batch2sens(batch_)